In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/model91/CIBMTRJoni(7).h5
/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv
/kaggle/input/equity-post-HCT-survival-predictions/data_dictionary.csv
/kaggle/input/equity-post-HCT-survival-predictions/train.csv
/kaggle/input/equity-post-HCT-survival-predictions/test.csv
/kaggle/input/model16/CIBMTRJoni(9).h5
/kaggle/input/model78/CIBMTRJoni(8).h5


In [2]:
df_test_orig=pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')

df_train=pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
df_dict=pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/data_dictionary.csv').loc[:56]

df_test=df_test_orig.copy()
# Procesar las columnas categóricas
for categorical_column in df_test[df_dict[df_dict['type'] == 'Categorical']['variable']].columns:
    if categorical_column != 'efs':
        # Generar un mapeo de categoría a número basado en el orden de categories_list
        categories_list = [x.replace("'", "").replace('"', "").replace(' nan','') for x in df_dict[df_dict['variable']==categorical_column]['values'].values[0][1:-1].split("' '")]
        print(categorical_column)
        category_to_number = {category: idx + 1 for idx, category in enumerate(categories_list)}
        print(category_to_number)
        # Mapear las categorías al dataframe
        df_test[categorical_column] = df_test[categorical_column].map(category_to_number)

# Rellenar valores faltantes
df_test.fillna(-100, inplace=True)

df_Xtest=df_test.set_index('ID')

dri_score
{'Intermediate': 1, 'High': 2, 'N/A - non-malignant indication': 3, 'N/A - pediatric\n High - TED AML case <missing cytogenetics': 4, 'TBD cytogenetics': 5, 'Low\n Intermediate - TED AML case <missing cytogenetics\n N/A - disease not classifiable Very high': 6, 'Missing disease status': 7}
psych_disturb
{'Yes': 1, 'No Not done': 2}
cyto_score
{'Intermediate': 1, 'Favorable': 2, 'Poor': 3, 'TBD Normal': 4, 'Other': 5, 'Not tested': 6}
diabetes
{'No': 1, 'Yes Not done': 2}
tbi_status
{'No TBI': 1, 'TBI + Cy +- Other': 2, 'TBI +- Other, <=cGy': 3, 'TBI +- Other, >cGy\n TBI +- Other, -cGy, single': 4, 'TBI +- Other, unknown dose\n TBI +- Other, -cGy, unknown dose': 5, 'TBI +- Other, -cGy, fractionated': 6}
arrhythmia
{'No Yes': 1, 'Not done': 2}
graft_type
{'Peripheral blood': 1, 'Bone marrow': 2}
vent_hist
{'No': 1, 'Yes': 2}
renal_issue
{'No Yes': 1, 'Not done': 2}
pulm_severe
{'No': 1, 'Yes Not done': 2}
prim_disease_hct
{'ALL': 1, 'MPN': 2, 'IPA': 3, 'AML': 4, 'MDS': 5, 'Othe

In [3]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Flatten, Concatenate, Lambda, LayerNormalization, Dense, MultiHeadAttention, Add, GlobalAveragePooling1D, Dropout, LSTM, ReLU, LeakyReLU, BatchNormalization

# Parámetros
embedding_dim = 64
drop = 0
num_heads = 4
key_dim = 64

# Entrada
inputs = tf.keras.Input(shape=(57,), name="Input")
# mask = tf.cast(tf.not_equal(inputs, -100), dtype=tf.float32)
mask = layers.Lambda(lambda x: tf.cast(tf.not_equal(x, -100), dtype=tf.float32))(inputs)

categorical_indices = df_dict.loc[:56,:][df_dict.loc[:56,:]['type']=='Categorical'].index  # Ejemplo de índices de columnas categóricas
categorical_vocab_sizes = (np.ones(categorical_indices.shape)*16).astype(int)  # Tamaños de vocabularios para cada columna categórica
# - Índices de columnas numéricas
numerical_indices = df_dict.loc[:56,:][df_dict.loc[:56,:]['type']=='Numerical'].index  # Ejemplo de índices de columnas numéricas




act = tf.nn.gelu

# Procesar columnas categóricas con embeddings
categorical_embeddings = []
for i, vocab_size in zip(categorical_indices, categorical_vocab_sizes):
    masked_input = inputs[:, i] * mask[:, i]
  # Aplicar la máscara
    embedding = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)(masked_input)
    embedding = layers.Lambda(lambda x: tf.expand_dims(x, 1))(embedding)
    embedding = layers.Dense(units=embedding_dim, activation=act)(embedding)
    embedding = layers.Dense(units=32, activation=act)(embedding)
    embedding = layers.Dense(units=8, activation=act)(embedding)
    # print(embedding.shape,'embedding')
    # print(type(embedding))
    categorical_embeddings.append(embedding)

categorical_embeddings = layers.Concatenate(axis=1)(categorical_embeddings)

# Procesar columnas numéricas con capas densas independientes
numerical_dense = []
for i in numerical_indices:
    masked_input = inputs[:, i] * mask[:, i]# Aplicar la máscara
    dense = layers.Dense(units=embedding_dim, activation=act)(layers.Lambda(lambda x: tf.expand_dims(x, -1))(masked_input))
    dense = layers.Lambda(lambda x: tf.expand_dims(x, 1))(dense)

    dense = layers.Dense(units=embedding_dim, activation=act)(dense)
    dense = layers.Dense(units=32, activation=act)(dense)
    dense = layers.Dense(units=8, activation=act)(dense)
    # print(type(dense))
    numerical_dense.append(dense)

numerical_dense = layers.Concatenate(axis=1)(numerical_dense)

# Concatenar todas las representaciones
combined_features = layers.Concatenate(axis=1)([categorical_embeddings, numerical_dense])

# Expandir la máscara para MultiHeadAttention
mask2 = layers.Lambda(lambda x: tf.repeat(tf.expand_dims(x, -1), repeats=8, axis=-1))(mask)
# combined_features = combined_features * mask2
combined_features = layers.Lambda(lambda x: tf.convert_to_tensor(x[0] * x[1]))([combined_features, mask2])


# attention_mask = tf.expand_dims(mask, 1) * tf.expand_dims(mask, 2)
attention_mask = layers.Lambda(lambda x: tf.convert_to_tensor(tf.expand_dims(x, 1) * tf.expand_dims(x, 2)))(mask)



num_heads = 4
key_dim = embedding_dim

# Primera capa de atención
attention_input = combined_features
attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(
    query=combined_features, value=attention_input, key=attention_input, attention_mask=attention_mask
)

out1 = layers.Add()([attention_input, attention_output])
ffn_output = layers.Dense(key_dim, activation=act)(out1)
ffn_output = layers.Dense(attention_input.shape[-1])(ffn_output)
attention_output = layers.Add()([out1, ffn_output])

# Segunda capa de atención
attention_input = attention_output
attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(
    query=attention_output, value=attention_output, key=attention_output, attention_mask=attention_mask
)

out1 = layers.Add()([attention_input, attention_output])
ffn_output = layers.Dense(key_dim, activation=act)(out1)
ffn_output = layers.Dense(attention_input.shape[-1])(ffn_output)
attention_output = layers.Add()([out1, ffn_output])

# Tercera capa de atención
attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(
    query=attention_output, value=attention_output, key=attention_output, attention_mask=attention_mask
)

out1 = layers.Add()([attention_input, attention_output])
ffn_output = layers.Dense(key_dim, activation=act)(out1)
ffn_output = layers.Dense(attention_input.shape[-1])(ffn_output)
attention_output = layers.Add()([out1, ffn_output])

# Cuarta capa de atención
attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(
    query=attention_output, value=attention_output, key=attention_output, attention_mask=attention_mask
)

out1 = layers.Add()([attention_input, attention_output])
ffn_output = layers.Dense(key_dim, activation=act)(out1)
ffn_output = layers.Dense(attention_input.shape[-1])(ffn_output)
attention_output = layers.Add()([out1, ffn_output])

# Flatten y capas densas
attention_output = layers.Flatten()(attention_output)
attention_output = layers.Dense(64, activation=act)(attention_output)
attention_output = layers.Dense(32, activation=act)(attention_output)

output = layers.Dense(1, activation="linear", name="Output")(attention_output)

# Crear el modelo
model = tf.keras.Model(inputs=inputs, outputs=output, name="Categorical_Numerical_Model")





In [4]:
# import requests
# from tensorflow.keras.models import load_model

# # ID del archivo
# url = 'https://drive.google.com/uc?export=download&id=1pOdGoPMMJcCuV7ue_9_jpFLX0SMlI0Up'

# response = requests.get(url)
# with open('modelo.h5', 'wb') as f:
#     f.write(response.content)

# model.load_weights('modelo.h5')

# model.save('model.h5')


In [5]:
model.load_weights('/kaggle/input/model16/CIBMTRJoni(9).h5')

In [6]:
efs_time_pred=model.predict(df_Xtest.values)

# efs_time_pred = np.nan_to_num(efs_time_pred, nan=0.0, posinf=10, neginf=-10)

I0000 00:00:1734009864.998369      68 service.cc:145] XLA service 0x7b4be4017b30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734009864.998430      68 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1734009864.998434      68 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1734009867.593426      95 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_289', 24 bytes spill stores, 24 bytes spill loads



1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step


I0000 00:00:1734009869.867690      68 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [7]:
risk_scores = efs_time_pred 

In [8]:
# sub=pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv')

# sub['ID']=df_Xtest.index
# sub['prediction']=risk_scores[:,0]
# Crear DataFrame para la submisión
sub = pd.DataFrame({
    'ID': df_Xtest.index,  # Asegúrate de usar la misma columna ID que en el conjunto de prueba
    'prediction': risk_scores[:,0]
})

In [9]:
sub.to_csv('submission.csv', index=False)


In [10]:
sub

,ID,prediction
0,28800,90.088753
1,28801,-15.444108
2,28802,65.505821
